# 2: ``PyTorch`` Model Training

* Time to run the cells: 
    * ~ 5 mins for 5 epochs, 1 exercise, 1 subset

Again, inspect `settings.py`. It is mostly the `ML_SETTINGS` that is relevant to
this notebook, though teh `RASCAL_HYPERS` and some options from `DATA_SETTINGS`
will be used too. `ML_SETTINGS` contains settings and hyperparameters for the
training procedure.

Create a directory at the relative path
`/rho_learn/docs/example/azoswitch/runs/`. This is where simulations will be run
and results stored.

In this notebook we will see how to construct custom torch models that integrate
with equistore, both linear and nonlinear. Also, we will construct a custom
torch loss function `CoulombLoss`, again interfaced with equistore. After
checking the equivariance condition of our structural representation and model,
we will do some model training.

## Define training settings

In [ ]:
import os
import pprint
import ase.io
import numpy as np
import torch

import chemiscope
import equistore

import rholearn
from rholearn import io, features, loss, models, pretraining, spherical, training, utils
from settings import RASCAL_HYPERS, DATA_SETTINGS, ML_SETTINGS

In [ ]:
# Create simulation run directory and save simulation
io.check_or_create_dir(ML_SETTINGS["run_dir"])
with open(os.path.join(ML_SETTINGS["run_dir"], "ml_settings.txt"), "a+") as f:
    f.write(f"ML Settings:\n{pprint.pformat(ML_SETTINGS)}\n\n")

# IMPORTANT! - set the torch default dtype
torch.set_default_dtype(ML_SETTINGS["torch"]["dtype"])

## Linear Model

In [ ]:
in_train = io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "in_train.npz"), **ML_SETTINGS["torch"]
)
out_train = io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "out_train.npz"), **ML_SETTINGS["torch"]
)

Using the ``EquiModelGlobal?`` command, we can see the arguments the global
model class takes upon initiliazation. The first 4 args are required to build
linear model, specifying the model type, in this case "linear", the keys of the
TensorMap that the model is built for, and the properties/features labels for
each of the blocks. These are required to map a number of input features to
output features for each block

In [ ]:
models.EquiModelGlobal?

Now we can build a linear model and, using the ``model.models`` attribute, print
out 2 of the local models that exist for the first 2 blocks, indexed by keys
``(0, 1)`` and ``(1, 1)``. You can see that the number of in a nf out features
of the linear layers are different, and that the model for the invariant block
uses a bias, whereas the one for the covariant ($\lambda = 1$) block doesn't.

In [ ]:
linear_model = models.EquiModelGlobal(
    "linear",
    keys=in_train.keys,
    in_feature_labels={key: block.properties for key, block in in_train},
    out_feature_labels={key: block.properties for key, block in out_train},
)
list(linear_model.models.items())[:2]

## Nonlinear Model

![nonlinear_model_forward](../figures/nonlinear_architecture.png)

In addition to a linear model, a nonlinear model is also implemented in
``rholearn``. As in the global linear model, the global nonlinear model is a
collection of individual local models applied to each block. Each local model
makes a prediction on a given equivariant (i.e. either invariant or covariant)
block of the input TensorMap, indexed by a key.

The nonlinear local model architecture is shown in the figure below. Predictions
are made on an equivariant (blue) block, using its associated invariant to act
as a nonlinear multiplier. For instance, the equivariant block for Carbon,
$\lambda = 3$ is passed to the forward method along with the invariant ($\lambda
= 0$) block for Carbon. The equivariant is passed through a linear model and the
invariant through a neural network of arbitrary architecture. Then, element-wise
multiplication of the two blocks is performed before passing the result through
a final linear output layer to get the electron density prediction.

Applying nonlinear transformations to only the invariant ensure that
equivariance isn't broken. Upon element-wise multiplication, the component
vectors of the equivariant block are multiplied by a vector of constant size
(thanks to the h-stacking of the invariant, botoom-right of the figure), thus
retaining equivariance.

Performing such operations within a single custom PyTorch ``forward()`` method
allows the operations to be tracked, and therefore the gradientsto be
calculated. This means that model training involves optimization of the weights
of all weights and biases seen below - in the linear input layer applied to the
equivariant, all layers of the neural network applied to the invariant, and the
linear output layer applied to the mixed block.

Let's build a global model by hand and look at just 2 of the individual local
models. As the keys of the TensorMap are ``('spherical_harmonics_l',
'species_center')``, there exists one invariant block for each chemical species
(i.e. ``species_center``: H (1), C (6), N (7), O (8), S (16)). As explained
above, these invariants are used as nonlinear multiplier to the equivariant
blocks, so the size of their features need to be passed to the model in the
``in_invariant_features`` during initialization. In the figure above, these
values correspond to $q_{\text{in}}^{\text{inv}}$ (bottom left of the figure).

The model architecture can also be controlled. The ``activation_fn`` to use in
alternating layers between linear layers can be specified, choosing from "Tanh",
"GELU", or "SiLU". The length of the list arg ``hidden_layer_widths`` controls
the number of pairs of hidden linear layers after the first input layer appear
in the neural network. For $n$ hidden linear layers, there will be $n-1$
nonlinear activation functions. The values in the list control the width
of these layers.

In the cell below, we are initializing the neural network to have 3 hidden
linear layers of widths 8, 8, and 16, with 2 nonlinear activation functions
sandwiched between them. Run the cell and look for the ``(invariant_nn)``
``Sequential`` layer, containing alternating Linear and SiLU functions. Note
also how for the ``EquiLocalModel`` for key ``('spherical_harmonics_l',
'species_center')`` == ``(0, 1)`` a bias is used on the input and output linear
layers, but for local model ``(1, 1)`` a bias isn't used. This is because for
covariant blocks ($\lambda > 0$) covariance is broken by applying a bias, but
for invariants it isn't. A bias is applied **in the neural network layers** for
local models, however, as only the invariant blocks supporting the forward
method are passed through the NN.

In [ ]:
np.unique(in_train.keys["species_center"])

In [ ]:
in_invariant_features_by_species = {
    specie: len(in_train.block(spherical_harmonics_l=0, species_center=specie).properties)
    for specie in np.unique(in_train.keys["species_center"])
}
in_invariant_features = {
    key: in_invariant_features_by_species[key[in_train.keys.names.index("species_center")]]
    for key in in_train.keys
}
nonlinear_model = models.EquiModelGlobal(
    "nonlinear",
    keys=in_train.keys,
    in_feature_labels={key: block.properties for key, block in in_train},
    out_feature_labels={key: block.properties for key, block in out_train},
    in_invariant_features=in_invariant_features,
    hidden_layer_widths=[8, 8, 16],
    activation_fn="SiLU",
)

list(nonlinear_model.models.items())[:2]

## Checking the Equivariance Condition

Before going any further, it is important that we check that our structural
representations and machine learning models are equivariant.

In order for a structural representation to be equivariant, the irreducible
spherical components that comprise it must transform like spherical harmonics.

Spherical harmonics have known behviour under rotations, such that any spherical
component $\mu$ of order $\lambda$ transforms into new component $\mu'$ according to the
action of the Wigner D-Matrix of order $\lambda$, $D^{\lambda}_{\mu\mu'}$, which
is constructed for a given arbitrary rotation matrix in Cartesian space.

In order to check that our $\lambda$-SOAP feature vector is equivariant, we can
run the following test:

1. Take an ``.xyz`` file of a given structure in the training set
2. Build an ASE frame of this structure
3. Generate a random cartesian rotation matrix
4. Rotate the xyz coordinates of the structure according to the random rotation
   matrix, storing the rotated structure in a new ASE frame
5. Generate a $\lambda$-SOAP representation for the unrotated and rotated
   structures
6. For each $\lambda$ channel of the representation of the unrotated structure,
   extract a selection of $(2\lambda + 1)$-sized irreducible spherical component (ISC)
   vectors.
7. Rotate each of these ISC vectors using the Wigner D-Matrix constucted
   according to cartesian rotation matrix defined in step 3
8. Check for exact equivalence between the rotated ISC vectors of the unrotated
   structure and the corresponding ISC vectors of the rotated structure.

First, let's load a random structure from the training set, construct rotated
and unrotated ASE frames, and visualize them using chemiscope. By moving the
slider, you can see that they are rigidly rotated versions of eachother.

In [ ]:
# Pick a random index between 0 and 10 as the test structure
structure_idx = np.random.randint(0, 10)

# Load the xyz file corresponding to this structure
with open(os.path.join(DATA_SETTINGS["data_dir"], "molecule_list.dat"), "r") as molecule_list:
    structure_xyz = molecule_list.read().splitlines()[structure_idx]

# Read xyz file into an ASE frame
unrotated = ase.io.read(os.path.join(DATA_SETTINGS["data_dir"], "xyz", structure_xyz))

# Generated a randomly rotated copy of the ASE frame
rotated, (alpha, beta, gamma) = spherical.rotate_ase_frame(unrotated)

# Visualize the frames using chemiscope
chemiscope.show([unrotated, rotated], mode="structure")

Now generate $\lambda$-SOAP representations of the rotated and unrotated
structures and pass these through a function that checks for equivariance.

In [ ]:
# Generate lambda-SOAP descriptors. We want to do this individually for the
# rotated and unrotated structures to keep the structure indices consistent
lsoap_unrotated = features.lambda_soap_vector(
    [unrotated], RASCAL_HYPERS, even_parity_only=True, neighbor_species=[1, 6, 7, 8, 16]
)
lsoap_rotated = features.lambda_soap_vector(
    [rotated], RASCAL_HYPERS, even_parity_only=True, neighbor_species=[1, 6, 7, 8, 16]
)

# Convert tensors to torch
lsoap_unrotated = utils.tensor_to_torch(lsoap_unrotated, **ML_SETTINGS["torch"])
lsoap_rotated = utils.tensor_to_torch(lsoap_rotated, **ML_SETTINGS["torch"])

# Perform the equivariance check - this returns a bool. Use the rotation matrix
# previosuly defined to produce the lambda-SOAP of the rotated and unrotated
# structures
is_equi = spherical.check_equivariance(
    lsoap_unrotated,
    lsoap_rotated,
    lmax=RASCAL_HYPERS["max_angular"],
    alpha=alpha,
    beta=beta,
    gamma=gamma,
    n_checks_per_block=5000,
)
if is_equi:
    print("Our lambda-SOAP is equivariant!")
else:
    print("Oops, our lambda-SOAP is not equivariant...")

We also need to check that our model is equivariant. When passing input tensors
through a model, certain tensor operations contained within the model
architecture (no matter how simple or complex) can break equivariance. It would
be a shame to go through the effort of generating an equivariant representation
if something as simple as applying a bias in our linear model breaks
equivariance!

In order for our model to be equivariant, it must satisfy the equivariant
condition: 

$\hat{R} y(A) = y(\hat{R} A)$

where $\hat{R}$ is an arbitrary rotation matrix of the SO(3) group, $A$ is a
trial structural representation, and $y(A)$ is the output property (i.e.
electron density) of the model, predicting on structure $A$.

In plainer words, the condition states (under the assumption that our structural
representation is equivariant) that our model is equivariant if the property
(electron density) we predict on an unrotated structure, subsequently rotated is
**exactly equivalent** to the property we get if predict on the rotated
structure, if the rotation matrix used in both is equal.

We can therefore construct a test in the following way, using the $\lambda$-SOAP
representations for the rotated and unrotated structures generated above:

1. Pass the $\lambda$-SOAP representation of the unrotated structure through the
   ML model to generate a predicted electron density.
2. Do the same for the $\lambda$-SOAP representation of the rotated structure.
3. For each $\lambda$ channel of the electron density of the unrotated
   structure, extract a selection of $(2\lambda + 1)$-sized irreducible
   spherical component (ISC) vectors.
4. Rotate each of these ISC vectors using the Wigner D-Matrix constucted
   according to cartesian rotation matrix defined above
5. Check for exact equivalence between the rotated ISC vectors of the unrotated
   electron density and the corresponding ISC vectors of the rotated electron
   density.

First, we need to do some cleaning (similar to as we did in the first notebook)
and padding of the TensorMaps to make the dimensions consistent - don't worry
too much about this. The, predict the electron density for the rotated and
unrotated structures, using the linear and nonlinear models.

In [ ]:
from equistore import Labels

# There may be some blocks in the training set that are not present in the
# validation structure; pad the validation structure with empty blocks
lsoap_rotated = utils.pad_with_empty_blocks(lsoap_rotated, in_train)
lsoap_unrotated = utils.pad_with_empty_blocks(lsoap_unrotated, in_train)

# Pass the rotated and unrotated structures through the linear model
out_pred_linear_unrot = linear_model(lsoap_unrotated)
out_pred_linear_rot = linear_model(lsoap_rotated)

# Pass the rotated and unrotated structures through the nonlinear model
out_pred_nonlin_unrot = nonlinear_model(lsoap_unrotated)
out_pred_nonlin_rot = nonlinear_model(lsoap_rotated)

Perform the equivariance check on both the linear and nonlinear model.

In [ ]:
# Perform the equivariance check on the linear and nonlinear models
for i, (unrot, rot) in enumerate(
    [
        (out_pred_linear_unrot, out_pred_linear_rot),
        (out_pred_nonlin_unrot, out_pred_nonlin_rot),
    ]
):
    is_equi = spherical.check_equivariance(
        unrot,
        rot,
        lmax=RASCAL_HYPERS["max_angular"],
        alpha=alpha,
        beta=beta,
        gamma=gamma,
        n_checks_per_block=None,  # None checks on all ISC vectors
    )
    if is_equi:
        print(f"Our {['linear', 'nonlinear'][i]} model is equivariant!")
    else:
        print(
            f"Oops, something in our {['linear', 'nonlinear'][i]} model is breaking equivariance..."
        )

Good stuff! Now that we've performed those checks, let's do some model training.

## Construct ``torch`` objects used in training

Prior to model training, it can be beneficial to pre-initialize and store some
of the torch objects that will be used, especially if some pre-processing
calculations can be performed.

### ML Model

PyTorch model training is based on torch tensor operations. In order to
interface with equistore and allow tracking of all the metadata useful in
atomistic ML, custom model classes have been built in ``rholearn`` to allow
predictions to be made on TensorMaps as a whole. The class ``EquiModelGlobal``
stores individual models for each input/output block in the data, as seen above.

### Loss Function

A function that calculates a difference metric between a predicted (or 'input')
and reference (or 'target') tensor. At the torch-equistore interface this is a
custom torch module that calculates this difference on the TensorMap level.

Currently implemented are the ``MSELoss`` (otherwise called L2 loss) and the
``CoulombLoss`` metrics. 

As detailed in the paper [__"Impact of quantum-chemical metrics on the machine
learning prediction of electron
density"__](https://aip.scitation.org/doi/10.1063/5.0055393), use of a
physically-inspired loss function such as the Coulomb repulsion metric can lead
to better model performance when predicting properties derived from the electron
density.

While the MSE loss needs no-preprocessing or initialization, when using a
CoulombLoss the speed of simulations can benefit greatly from pre-processing.
The Coulomb matrices can be distilled to contain data only for structures present
in the training and test data sets they will evaluate the loss for. Especially
as multiple training exercises on various subsets will be performed,
initializing and storing both a train loss and test loss object for each
training subset, to be loaded at runtime, can speed things up.

provided can be heavly pre-processed prior to use at
runtime of model training. The custom torch module ``CoulombLoss`` pre-processes

### Optimizer

An algorithm, such as stochastic gradient descent (SGD) or LBFGS that performs
gradient descent on the loss landscape with respect to the model parameters.
This is generally a lightweight object that doesn't need to be pre-constructed.

## Initializing a CoulombLoss object

Let's build train and test CoulombLoss objects.

In [ ]:
# Load the output train data from the exercise 0 subset 0 training directory
out_train = rholearn.io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "exercise_0", "subset_0", "out_train.npz"),
    **ML_SETTINGS["torch"]
)
# Load the output test data - is independent of training subset
out_test = rholearn.io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "out_test.npz"), **ML_SETTINGS["torch"]
)

# Load the Coulomb matrices from file
coulomb_matrices = rholearn.io.load_tensormap_to_torch(
    DATA_SETTINGS["coulomb_metrics"], **ML_SETTINGS["torch"]
)

# Construct the train and test loss CoulombLoss functions
train_loss_fn = loss.CoulombLoss(coulomb_matrices, output_like=out_train)
test_loss_fn = loss.CoulombLoss(coulomb_matrices, output_like=out_test)

# Construct the MSELoss - no special initialization needed
mse_loss_fn = loss.MSELoss(reduction="sum")

In [ ]:
# Load the imput train and test data
in_train = rholearn.io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "exercise_0", "subset_0", "in_train.npz"),
    **ML_SETTINGS["torch"],
)
in_test = rholearn.io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "in_test.npz"),
    **ML_SETTINGS["torch"],
)

# Make a prediction with the untrained linear model
out_train_pred = linear_model(in_train)
out_test_pred = linear_model(in_test)

# Calculate and print losses
print("linear model")
print(
    f"    CoulombLoss: train = {train_loss_fn(input=out_train_pred, target=out_train)},"
    + f" test = {test_loss_fn(input=out_test_pred, target=out_test)} Ha"
    + f"\n    MSELoss: train = {mse_loss_fn(input=out_train_pred, target=out_train)},"
    + f" test = {mse_loss_fn(input=out_test_pred, target=out_test)}"
)

# Make a prediction with the untrained nonlinear model, print losses
out_train_pred = nonlinear_model(in_train)
out_test_pred = nonlinear_model(in_test)
print("nonlinear model")
print(
    f"    CoulombLoss: train = {train_loss_fn(input=out_train_pred, target=out_train)},"
    + f" test = {test_loss_fn(input=out_test_pred, target=out_test)} Ha"
    + f"\n    MSELoss: train = {mse_loss_fn(input=out_train_pred, target=out_train)},"
    + f" test = {mse_loss_fn(input=out_test_pred, target=out_test)}"
)

Notice that losses are quite high. This is to be expected - these are untrained
models!

## Training a Linear Model

As every training subsets (whether belonging to the same or different
learning exercise) are independent, model training can be performed separately
and in principle in parallel. Here we will perform subset training sequentially.

First, define the training settings.

Now prepare each training subdirectory by contructing models and loss functions
in them, ready for training.

In [ ]:
# Pre-construct the appropriate torch objects (i.e. models, loss fxns)
pretraining.construct_torch_objects(
    DATA_SETTINGS,
    ML_SETTINGS,
    coulomb_path=DATA_SETTINGS["coulomb_metrics"],
)

Inspect the directory structure in the ``simulations/`` folder - it mirrors the
nested directory structure of the ``data/`` folder, but contains only torch
objects corresponding to the torch model ``model.pt``, the coulomb loss function
object ``loss_fn.pt`` and that of the test loss function ``loss_fn_test.pt``.

As the test data is not dependent on the training subset, this can be loaded
first. The torch settings (i.e. requires_grad, device, dtype) from the settings
dict are used to load the TensorMaps to torch.

In [ ]:
# Load the test data, which is independent of the training subdirectory
in_test = rholearn.io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "in_test.npz"), **ML_SETTINGS["torch"]
)
out_test = rholearn.io.load_tensormap_to_torch(
    os.path.join(DATA_SETTINGS["data_dir"], "out_test.npz"), **ML_SETTINGS["torch"]
)

Now we iterate over the exercises and subsets and train models for each subset
sequentially. Every epoch, the train and test losses are written to `log.txt` in
the training directory. Inspect the file during runtime and observe the
minimization of the loss.

In [ ]:
# Runtime for 10 epochs, 2 exercises, 3 subsets:
# linear ~ 15 min
# nonlinear ~ 20 min
from itertools import product

exercises = [0]
subsets = [0, 1, 2]

for exercise, subset in product(exercises, subsets):

    # Define the training subdirectory
    train_rel_dir = f"exercise_{exercise}/subset_{subset}"
    train_run_dir = os.path.join(ML_SETTINGS["run_dir"], train_rel_dir)

    # Load training data and torch objects
    data, model, loss_fn, optimizer = pretraining.load_training_objects(
        train_rel_dir, DATA_SETTINGS["data_dir"], ML_SETTINGS, ML_SETTINGS["training"]["restart_epoch"]
    )

    # Unpack the data
    in_train, in_test, out_train, out_test = data

    # Execute model training
    print(f"\nTraining in subdirectory {train_rel_dir}")
    training.train(
        in_train=in_train,
        out_train=out_train,
        in_test=in_test,
        out_test=out_test,
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        n_epochs=ML_SETTINGS["training"]["n_epochs"],
        save_interval=ML_SETTINGS["training"]["save_interval"],
        save_dir=train_run_dir,
        restart=ML_SETTINGS["training"]["restart_epoch"],
    )

## Training a Nonlinear Model

Now we can use the same notebook to train a nonlinear model, by changing only a
few lines of code. Make the following changes to `ML_SETTINGS` in `settings.py`.

1. Change the run directory:

    ``"run_dir": os.path.join(RHOLEARN_DIR, "docs/example/azoswitch/runs/02_nonlinear"),``


2. Change model type to "nonlinear" and uncomment the neural network args:
  
        ```
        "type": "nonlinear",
        "args": {
            "hidden_layer_widths": [4, 4],
            "activation_fn": "SiLU"
        },
        ```

Then run the subsequent cells again, in order.